In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot import ChatBot, load_api_key
from OktSentiment import Sentiment
from tarot import TarotBot
from refer_text import ResponseSearch

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
respon_search = ResponseSearch('c:/Users/SMHRD/최종_참조자료.csv',model_name="jhgan/ko-sroberta-multitask")
chatbot = ChatBot(api_key, 'c:/Users/SMHRD/원인-대처종합.csv')  # CSV 파일 경로를 적절히 수정하세요
#C:\Users\SMHRD\Documents\카카오톡 받은 파일\원인-대처종합.csv //c:/Users/SMHRD/원인-대처종합.csv
# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    data = request.json
    cards = data.get("cards")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ",data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}),400
    
    key_path = "c:/Users/SMHRD/api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if user_input:
            print(user_input)
            response = tarot.interpret_cards(user_input,cards)
        else:
            response = tarot.general_reading(user_select,cards)
        print("response: ",response.get_json())
        return response

    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}),500

if __name__ == "__main__":
    app.run(port=7000)

c:\Users\SMHRD\MindRoad\flask_modules\chatbot.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  self.model = ChatOpenAI(temperature=0.7, model_name="gpt-4", openai_api_key=api_key)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [07/Sep/2024 16:22:35] "OPTIONS /chatbot HTTP/1.1" 200 -


Received data: {'user_nick': '12', 'input': '명동으로 갈까요'}
Processing input: user_nick=12, user_input=명동으로 갈까요


c:\Users\SMHRD\MindRoad\flask_modules\chatbot.py:29: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=self.model, prompt=template)
c:\Users\SMHRD\MindRoad\flask_modules\chatbot.py:30: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain.run(user_input=user_input).strip()


Is problem flag: False
고민이 아님


127.0.0.1 - - [07/Sep/2024 16:22:44] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [07/Sep/2024 16:23:28] "OPTIONS /api/interpret HTTP/1.1" 200 -


recived data:  {'user_select': '오늘의 운세', 'cards': ['King of Cups', 'Knight of Cups', 'Five of Pentacles']}


127.0.0.1 - - [07/Sep/2024 16:23:31] "POST /api/interpret HTTP/1.1" 200 -


response:  {'answer': {'cards': 'King of Cups, Knight of Cups, Five of Pentacles', 'text': '- King of Cups: 감정의 통제와 지혜로운 판단을 의미합니다.\n- Knight of Cups: 로맨틱하고 감성적인 제안이나 소식을 상징합니다.\n- Five of Pentacles: 어려움과 결핍, 특히 재정적인 문제를 나타냅니다.\n\n오늘의 운세를 종합해보면, 감정적으로 안정된 상태에서 중요한 결정을 내릴 수 있는 날입니다. 누군가 로맨틱하거나 감성적인 접근을 시도할 수 있으며, 이는 당신에게 긍정적인 영향을 미칠 것입니다. 하지만 재정적인 문제나 자원 부족에 대한 우려가 있을 수 있으니 신중하게 대처하세요. 주변 사람들에게 도움을 요청하는 것도 좋은 선택이 될 수 있습니다.', 'user_select': '오늘의 운세'}}
